In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import DB_library as DBL
import numpy as np
import os

sns.set_theme()

In [2]:
path_to_XDATCAR = '../../ICMAB/Data/Li-based/Li10GeS2P12/AIMD/stoichiometric/650K'

In [3]:
atomic_numbers = np.load('atomic_numbers.npy', allow_pickle='TRUE').item()

### Extract data from INCAR file at path_to_XDATCAR

In [4]:
def read_INCAR(path_to_simulation):
    """Reads VASP INCAR files. It is always expected to find these parameters.
    Read VASP INCAR Settings

    Args:
        path_to_simulation (str): Path to the simulation directory.

    Returns:
        tuple: A tuple containing delta_t and n_steps.
    """
    
    # Predefining the variable, so later we check if they were found
    
    delta_t = None
    n_steps = None
    
    # Loading the INCAR file
    
    if not os.path.exists(f'{path_to_simulation}/INCAR'):
        exit('INCAR file is not available.')
    
    with open(f'{path_to_simulation}/INCAR', 'r') as INCAR_file:
        INCAR_lines = INCAR_file.readlines()
    
    # Looking for delta_t and n_steps
    
    for line in INCAR_lines:
        split_line = line.split('=')
        if len(split_line) > 1:  # Skipping empty lines
            label = split_line[0].split()[0]
            value = split_line[1].split()[0]
            
            if   label == 'POTIM':  delta_t = float(value)
            elif label == 'NBLOCK': n_steps = float(value)
    
    # Checking if they were found
    
    if (delta_t is None) or (n_steps is None):
        exit('POTIM or NBLOCK are not correctly defined in the INCAR file.')
    return delta_t, n_steps

INCAR_delta_t, INCAR_n_steps = read_INCAR(path_to_XDATCAR)
INCAR_delta_t, INCAR_n_steps

(1.5, 10.0)

### Extract data from XDATCAR file at path_to_XDATCAR

In [5]:
# Import the XDATCAR file
XDATCAR_lines = [line for line in open(f'{path_to_XDATCAR}/XDATCAR') if line.strip()]

# Extract initial XDATCAR data
compound = XDATCAR_lines[0][:-1]
scale    = float(XDATCAR_lines[1])

lattice_vectors = np.array([line.split() for line in XDATCAR_lines[2:5]], dtype=float)
lattice_vectors *= scale  # Use scaling

composition   = XDATCAR_lines[5].split()
concentration = np.array(XDATCAR_lines[6].split(), dtype=int)

n_atoms = np.sum(concentration)  # Number of particles within the simulation box

print_name = ' '.join(composition)
print(f'Compound: {compound}')
print(f'Composition: {print_name}')
print(f'Concentration: {concentration}')

# Shape the configurations data into the positions attribute
direct_coordinates = np.array([line.split() for line in XDATCAR_lines[8:] if not line.split()[0][0].isalpha()], dtype=float)

direct_coordinates = direct_coordinates.ravel().reshape((-1, n_atoms, 3))  # (n_conf, n_atoms, 3) tensor
n_conf = direct_coordinates.shape[0]  # Number of configurations or simulation steps

print(f'Number of configurations: {n_conf}')

Compound: Li10GeS2P12
Composition: Li Ge P S
Concentration: [80  8 16 96]
Number of configurations: 9966


### Identify diffusive particles

In [6]:
diffusive_particle = 'Li'

for i in range(n_atoms):
    if composition[i] == diffusive_particle:
        diffusive_idx = i
        break

concentration_cumsum = np.insert(np.cumsum(concentration), 0, 0)

# Exact indexes of the diffive particles
diffusive_indexes = np.arange(concentration_cumsum[diffusive_idx],
                              concentration_cumsum[diffusive_idx+1],
                             dtype=int)

# Index where the diffive particles start
diffusive_indexes_start = diffusive_indexes[0]

# Number of diffive particles
n_diffusive_indexes = len(diffusive_indexes)

diffusive_indexes

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79])

### Revert periodic boundary conditions and pass to cartesian coordinates

In [7]:
# Get the variation in positions
dpos = np.diff(direct_coordinates, axis=0)

# Revert periodic boundary condition
dpos[dpos > 0.5]  -= 1.0
dpos[dpos < -0.5] += 1.0

# Pass to cartesian
for i in range(n_conf-1):
    dpos[i] = np.dot(dpos[i], lattice_vectors)

# Copy direct to cartesian and pass first frame (configuration)
cartesian_coordinates    = direct_coordinates.copy()
cartesian_coordinates[0] = np.dot(cartesian_coordinates[0], lattice_vectors)

# Expand dimensions and sum every dpos
expanded_dimensions   = np.expand_dims(cartesian_coordinates[0], 0)
cartesian_coordinates = np.concatenate([expanded_dimensions, dpos], axis=0)
cartesian_coordinates = np.cumsum(cartesian_coordinates, axis=0)

### Generate tensor

In [8]:
differences_tensor_mean = np.zeros((n_conf, n_atoms, 3))
differences_tensor_std  = np.zeros((n_conf, n_atoms, 3))

# We vectorize in terms of n_atoms (only possibility here)
for delta_t in np.arange(1, n_conf):  # delta_t = 0 gives 0 by definition
    # Number of windows which are used for screening distances
    n_windows = n_conf - delta_t
    
    # Extract mean and standard deviation
    delta_t_distance = np.zeros((n_atoms, n_windows, 3))
    for t_0 in range(n_windows):
        # Distance between two configurations of a same particle
        # td (atom_i, dim_i) = cc (t_0 + delta_t, atom_i, dim_k) - cc (t_0, atom_i, dim_k)
        temporal_dist = cartesian_coordinates[t_0 + delta_t] - cartesian_coordinates[t_0]

        # Add Euclidean distance
        delta_t_distance[:, t_0] = temporal_dist
    
    # Generate mean over windows (axis=1)
    differences_tensor_mean[delta_t] = np.mean(delta_t_distance, axis=1)
    
    # Generate std over windows (axis=1)
    differences_tensor_std[delta_t] = np.std(delta_t_distance, axis=1)


KeyboardInterrupt



### Generate MSD

In [ ]:
# Define the array of time simulation in pico-seconds
delta_t_array = np.arange(n_conf) * (INCAR_n_steps * INCAR_delta_t * 1e-3)

MSD_self     = np.zeros(n_conf)
MSD_distinct = np.zeros(n_conf)

n_self     = 0
n_distinct = 0

for i in np.arange(n_diffusive_indexes):
    idx_i = diffusive_indexes_start + i
    
    # Self product
    for dim_k in range(3):
        # MSD_self (delta_n_conf) = dtm (delta_n_conf, atom_i, dim_k) * dtm (delta_n_conf, atom_i, dim_k)
        MSD_self += (differences_tensor_mean[:, idx_i, dim_k] * differences_tensor_mean[:, idx_i, dim_k])
    n_self += 1
    
    for j in np.arange(i+1, n_diffusive_indexes):
        idx_j = diffusive_indexes_start + j
        
        # Distinct (cross) product
        for dim_k in range(3):
            # MSD_distinct (delta_n_conf) = dtm (delta_n_conf, atom_i, dim_k) * dtm (delta_n_conf, atom_j, dim_k)
            MSD_distinct += (differences_tensor_mean[:, idx_i, dim_k] * differences_tensor_mean[:, idx_j, dim_k])
        n_distinct += 1

MSD_self     /= n_self
MSD_distinct /= n_distinct

MSD_self, MSD_distinct

### Compute diffusion coefficient

In [ ]:
initial_point = None

x_fit = delta_t_array
yerr_fit = None  #yerr_fit = y[initial_point:]

if initial_point is None:
    initial_point = int(0.1 * n_conf)
else:
    initial_point = int(initial_point * n_conf)

y_fit  = MSD_self
_beta_ = DBL.weighted_regression(x_fit[initial_point:],
                                 y_fit[initial_point:],
                                 DBL.linear_function,
                                 yerr=yerr_fit).beta
coef_D_self = _beta_[1]

y_fit  = MSD_self+MSD_distinct
_beta_ = DBL.weighted_regression(x_fit[initial_point:],
                                 y_fit[initial_point:],
                                 DBL.linear_function,
                                 yerr=yerr_fit).beta
coef_D_full = _beta_[1]

coef_D_self, coef_D_full

### Plot MSD

In [ ]:
plt.plot(delta_t_array,
         MSD_self,
         label='Self')
plt.plot(delta_t_array,
         MSD_self+MSD_distinct,
         label='Self+distinct')

plt.xlabel(r'$\Delta t$ (ps)')
plt.ylabel(r'$MSD$ ($\mathregular{Å^2}$)')
plt.legend(loc='best')
plt.show()